In [1]:
!pip install transformers decord numpy torch


[notice] A new release of pip is available: 25.0.1 -> 25.1.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [3]:
# from transformers import VideoMAEForVideoClassification, VideoMAEImageProcessor
# from decord import VideoReader
# import numpy as np
# import torch

# # Load model and feature extractor
# model = VideoMAEForVideoClassification.from_pretrained(
#     r"F:\quest_digiflex\exp\tf\Tf100",  # or wherever your config.json and safetensors are stored
#     local_files_only=True
# )
# feature_extractor = VideoMAEImageProcessor.from_pretrained(
#     "facebook/timesformer-base-finetuned-k400"
# )

# # Set device
# device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
# model.to(device)
# model.eval()


ImportError: cannot import name 'TimesformerImageProcessor' from 'transformers' (C:\Users\prana\AppData\Local\Programs\Python\Python312\Lib\site-packages\transformers\__init__.py)

In [6]:
from transformers import TimesformerForVideoClassification, AutoImageProcessor
import torch

model = TimesformerForVideoClassification.from_pretrained(
    r"F:\quest_digiflex\exp\tf\Tf100",
    local_files_only=True,
    ignore_mismatched_sizes=True
)

processor = AutoImageProcessor.from_pretrained(
    "facebook/timesformer-base-finetuned-k400"
)

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)
model.eval()


Some weights of TimesformerForVideoClassification were not initialized from the model checkpoint at F:\quest_digiflex\exp\tf\Tf100 and are newly initialized because the shapes did not match:
- classifier.bias: found shape torch.Size([22]) in the checkpoint and torch.Size([400]) in the model instantiated
- classifier.weight: found shape torch.Size([22, 768]) in the checkpoint and torch.Size([400, 768]) in the model instantiated
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


preprocessor_config.json:   0%|          | 0.00/412 [00:00<?, ?B/s]

C:\Users\prana\AppData\Local\Programs\Python\Python312\Lib\site-packages\huggingface_hub\file_download.py:140: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\prana\.cache\huggingface\hub\models--facebook--timesformer-base-finetuned-k400. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)
Using a slow image processor as `use_fast` is unset and a slow processor 

TimesformerForVideoClassification(
  (timesformer): TimesformerModel(
    (embeddings): TimesformerEmbeddings(
      (patch_embeddings): TimesformerPatchEmbeddings(
        (projection): Conv2d(3, 768, kernel_size=(16, 16), stride=(16, 16))
      )
      (pos_drop): Dropout(p=0.0, inplace=False)
      (time_drop): Dropout(p=0.0, inplace=False)
    )
    (encoder): TimesformerEncoder(
      (layer): ModuleList(
        (0-11): 12 x TimesformerLayer(
          (drop_path): Identity()
          (attention): TimeSformerAttention(
            (attention): TimesformerSelfAttention(
              (qkv): Linear(in_features=768, out_features=2304, bias=True)
              (attn_drop): Dropout(p=0.0, inplace=False)
            )
            (output): TimesformerSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.0, inplace=False)
            )
          )
          (intermediate): TimesformerIntermediate(
            (dense

In [7]:
# from transformers import VideoMAEForVideoClassification, VideoMAEImageProcessor, AutoConfig
# from decord import VideoReader
# import numpy as np
# import torch
# import os

# # --- Load Model, Config, Feature Extractor ---
# model_dir = r"F:\quest_digiflex\exp\tf\model_82"

# # Make sure model_10.safetensor is renamed to model.safetensors before running this
# # model = VideoMAEForVideoClassification.from_pretrained(model_dir, local_files_only=True)
# feature_extractor = VideoMAEImageProcessor.from_pretrained("MCG-NJU/videomae-base")
# config = AutoConfig.from_pretrained(model_dir, local_files_only=True)
# id2label = config.id2label  # Automatically retrieved from saved config

# # Setup device
# device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
# model.to(device)
# model.eval()


VideoMAEForVideoClassification(
  (videomae): VideoMAEModel(
    (embeddings): VideoMAEEmbeddings(
      (patch_embeddings): VideoMAEPatchEmbeddings(
        (projection): Conv3d(3, 768, kernel_size=(2, 16, 16), stride=(2, 16, 16))
      )
    )
    (encoder): VideoMAEEncoder(
      (layer): ModuleList(
        (0-11): 12 x VideoMAELayer(
          (attention): VideoMAESdpaAttention(
            (attention): VideoMAESdpaSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=False)
              (key): Linear(in_features=768, out_features=768, bias=False)
              (value): Linear(in_features=768, out_features=768, bias=False)
              (dropout): Dropout(p=0.0, inplace=False)
            )
            (output): VideoMAESelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.0, inplace=False)
            )
          )
          (intermediate): VideoMAEIntermediate(
            (den

In [8]:
def predict_video_class(video_path, model, extractor, num_frames=16, id2label=None):
    vr = VideoReader(video_path)
    total_frames = len(vr)
    frame_indices = np.linspace(0, total_frames - 1, num_frames, dtype=int)
    frames = vr.get_batch(frame_indices).asnumpy()  # shape: (num_frames, H, W, 3)

    # Extract features
    inputs = extractor(list(frames), return_tensors="pt")
    pixel_values = inputs["pixel_values"].to(device)

    # Forward pass
    with torch.no_grad():
        outputs = model(pixel_values=pixel_values)
        logits = outputs.logits
        predicted_class = logits.argmax(-1).item()

    # Decode class
    if id2label:
        return id2label[predicted_class]
    return predicted_class


In [9]:
# video_path = r"F:\quest_digiflex\exp\tf\videomae-finetuned\Bodyweight Squats.mp4"

# # You must define this mapping (same used during training)
# id2label = {0: "jumping_jack", 1: "squat", 2: "pushup", ...}  # example

# predicted_class = predict_video_class(video_path, model, feature_extractor, id2label=id2label)
# print("Predicted class:", predicted_class)


SyntaxError: ':' expected after dictionary key (2496193448.py, line 4)

In [8]:
# from transformers import VideoMAEForVideoClassification, VideoMAEImageProcessor, AutoConfig
# from decord import VideoReader
# import numpy as np
# import torch
# import os

# # --- Load Model, Config, Feature Extractor ---
# model_dir = r"F:/quest_digiflex/exp/tf/videomae-finetuned"

# # Make sure model_10.safetensor is renamed to model.safetensors before running this
# model = VideoMAEForVideoClassification.from_pretrained(model_dir, local_files_only=True)
# feature_extractor = VideoMAEImageProcessor.from_pretrained("MCG-NJU/videomae-base")
# config = AutoConfig.from_pretrained(model_dir, local_files_only=True)
# id2label = config.id2label  # Automatically retrieved from saved config

# # Setup device
# device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
# model.to(device)
# model.eval()

# # --- Inference Function ---
# def predict_video_class(video_path, model, extractor, num_frames=16, id2label=None):
#     vr = VideoReader(video_path)
#     total_frames = len(vr)
    
#     if total_frames < num_frames:
#         raise ValueError(f"Video has only {total_frames} frames, but {num_frames} are required.")

#     frame_indices = np.linspace(0, total_frames - 1, num_frames, dtype=int)
#     frames = vr.get_batch(frame_indices).asnumpy()  # (num_frames, H, W, 3)

#     inputs = extractor(list(frames), return_tensors="pt")
#     pixel_values = inputs["pixel_values"].to(device)

#     with torch.no_grad():
#         outputs = model(pixel_values=pixel_values)
#         logits = outputs.logits
#         predicted_class = logits.argmax(-1).item()

#     return id2label[predicted_class] if id2label else predicted_class

# # --- Predict on Unseen Video ---
# video_path = r"F:\quest_digiflex\exp\tf\videomae-finetuned\Bodyweight Squats.mp4"  # Update this to your actual video path

# predicted_label = predict_video_class(video_path, model, feature_extractor, id2label=id2label)
# print("✅ Predicted Exercise Class:", predicted_label)


✅ Predicted Exercise Class: squat


In [10]:
# # --- Predict on Unseen Video ---
# video_path = r"F:\quest_digiflex\exp\tf\videomae-finetuned\test\pull Up\pull up_1.mp4"  # Update this to your actual video path

# predicted_label = predict_video_class(video_path, model, feature_extractor, id2label=id2label)
# print("✅ Predicted Exercise Class:", predicted_label)


✅ Predicted Exercise Class: t bar row


In [10]:
# # --- Predict on Unseen Video ---
# video_path = r"F:\quest_digiflex\exp\tf\videomae-finetuned\test\pull Up\pull up_2.mp4"  # Update this to your actual video path

# predicted_label = predict_video_class(video_path, model, feature_extractor, id2label=id2label)
# print("✅ Predicted Exercise Class:", predicted_label)


✅ Predicted Exercise Class: pull Up


In [11]:
# # --- Predict on Unseen Video ---
# video_path = r"F:\quest_digiflex\exp\tf\videomae-finetuned\test\push-up\push-up_1.mp4"  # Update this to your actual video path

# predicted_label = predict_video_class(video_path, model, feature_extractor, id2label=id2label)
# print("✅ Predicted Exercise Class:", predicted_label)


✅ Predicted Exercise Class: russian twist


In [12]:
# # --- Predict on Unseen Video ---
# video_path = r"F:\quest_digiflex\exp\tf\videomae-finetuned\test\push-up\video15.mp4"  # Update this to your actual video path

# predicted_label = predict_video_class(video_path, model, feature_extractor, id2label=id2label)
# print("✅ Predicted Exercise Class:", predicted_label)


✅ Predicted Exercise Class: push-up


In [9]:
import os
from glob import glob
import pandas as pd
from collections import Counter

# --- Test Directory ---
test_root = r"F:\quest_digiflex\exp\tf\videomae-finetuned\test"

# --- Store results ---
results = []

# --- Process each video ---
for actual_class in os.listdir(test_root):
    class_path = os.path.join(test_root, actual_class)
    if not os.path.isdir(class_path):
        continue

    video_files = glob(os.path.join(class_path, "*.mp4"))

    for video_path in video_files:
        try:
            predicted_label = predict_video_class(video_path, model, feature_extractor, id2label=id2label)
            video_name = os.path.basename(video_path)
            results.append({
                "video": video_name,
                "actual_class": actual_class,
                "predicted_class": predicted_label
            })
            print(f"✅ {video_name} | Actual: {actual_class} --> Predicted: {predicted_label}")
        except Exception as e:
            print(f"⚠️ Error processing {video_path}: {e}")
            continue

# --- Convert to DataFrame ---
df_results = pd.DataFrame(results)

# --- Accuracy Analysis ---
df_results["is_correct"] = df_results["actual_class"] == df_results["predicted_class"]

total_videos = len(df_results)
correct_preds = df_results["is_correct"].sum()
incorrect_preds = total_videos - correct_preds

# --- Most misclassified class ---
incorrect_df = df_results[df_results["is_correct"] == False]
most_misclassified_class = incorrect_df["actual_class"].value_counts().idxmax() if not incorrect_df.empty else None

# --- Output Summary ---
print("\n=== Prediction Summary ===")
print(df_results)

print("\n=== Statistics ===")
print(f"🎞️ Total videos processed     : {total_videos}")
print(f"✅ Correct predictions        : {correct_preds}")
print(f"❌ Incorrect predictions      : {incorrect_preds}")
if most_misclassified_class:
    print(f"⚠️ Most misclassified class   : {most_misclassified_class}")

# --- Optional: Save to CSV ---
# df_results.to_csv("video_predictions_with_accuracy.csv", index=False)


⚠️ Error processing F:\quest_digiflex\exp\tf\videomae-finetuned\test\barbell biceps curl\barbell biceps curl_1.mp4: name 'feature_extractor' is not defined
⚠️ Error processing F:\quest_digiflex\exp\tf\videomae-finetuned\test\barbell biceps curl\video1.mp4: name 'feature_extractor' is not defined
⚠️ Error processing F:\quest_digiflex\exp\tf\videomae-finetuned\test\bench press\bench press_1.mp4: name 'feature_extractor' is not defined
⚠️ Error processing F:\quest_digiflex\exp\tf\videomae-finetuned\test\bench press\bench press_2.mp4: name 'feature_extractor' is not defined
⚠️ Error processing F:\quest_digiflex\exp\tf\videomae-finetuned\test\bench press\video2.mp4: name 'feature_extractor' is not defined
⚠️ Error processing F:\quest_digiflex\exp\tf\videomae-finetuned\test\chest fly machine\chest fly machine_1.mp4: name 'feature_extractor' is not defined
⚠️ Error processing F:\quest_digiflex\exp\tf\videomae-finetuned\test\chest fly machine\chest fly machine_2.mp4: name 'feature_extractor' i

KeyError: 'actual_class'

In [10]:
# import os
# from glob import glob
# import pandas as pd
# from collections import Counter

# # --- Test Directory ---
# test_root = r"F:\quest_digiflex\exp\data\archive"

# # --- Store results ---
# results = []

# # --- Process each video ---
# for actual_class in os.listdir(test_root):
#     class_path = os.path.join(test_root, actual_class)
#     if not os.path.isdir(class_path):
#         continue

#     video_files = glob(os.path.join(class_path, "*.mp4"))

#     for video_path in video_files:
#         try:
#             predicted_label = predict_video_class(video_path, model, feature_extractor, id2label=id2label)
#             video_name = os.path.basename(video_path)
#             results.append({
#                 "video": video_name,
#                 "actual_class": actual_class,
#                 "predicted_class": predicted_label
#             })
#             print(f"✅ {video_name} | Actual: {actual_class} --> Predicted: {predicted_label}")
#         except Exception as e:
#             print(f"⚠️ Error processing {video_path}: {e}")
#             continue

# # --- Convert to DataFrame ---
# df_results = pd.DataFrame(results)

# # --- Accuracy Analysis ---
# df_results["is_correct"] = df_results["actual_class"] == df_results["predicted_class"]

# total_videos = len(df_results)
# correct_preds = df_results["is_correct"].sum()
# incorrect_preds = total_videos - correct_preds

# # --- Most misclassified class ---
# incorrect_df = df_results[df_results["is_correct"] == False]
# most_misclassified_class = incorrect_df["actual_class"].value_counts().idxmax() if not incorrect_df.empty else None

# # --- Output Summary ---
# print("\n=== Prediction Summary ===")
# print(df_results)

# print("\n=== Statistics ===")
# print(f"🎞️ Total videos processed     : {total_videos}")
# print(f"✅ Correct predictions        : {correct_preds}")
# print(f"❌ Incorrect predictions      : {incorrect_preds}")
# if most_misclassified_class:
#     print(f"⚠️ Most misclassified class   : {most_misclassified_class}")

# # --- Optional: Save to CSV ---
# # df_results.to_csv("video_predictions_with_accuracy.csv", index=False)


⚠️ Error processing F:\quest_digiflex\exp\data\archive\barbell biceps curl\barbell biceps curl_1.mp4: name 'feature_extractor' is not defined
⚠️ Error processing F:\quest_digiflex\exp\data\archive\barbell biceps curl\barbell biceps curl_10.mp4: name 'feature_extractor' is not defined
⚠️ Error processing F:\quest_digiflex\exp\data\archive\barbell biceps curl\barbell biceps curl_11.mp4: name 'feature_extractor' is not defined
⚠️ Error processing F:\quest_digiflex\exp\data\archive\barbell biceps curl\barbell biceps curl_12.mp4: name 'feature_extractor' is not defined
⚠️ Error processing F:\quest_digiflex\exp\data\archive\barbell biceps curl\barbell biceps curl_13.mp4: name 'feature_extractor' is not defined
⚠️ Error processing F:\quest_digiflex\exp\data\archive\barbell biceps curl\barbell biceps curl_14.mp4: name 'feature_extractor' is not defined
⚠️ Error processing F:\quest_digiflex\exp\data\archive\barbell biceps curl\barbell biceps curl_15.mp4: name 'feature_extractor' is not defined


KeyError: 'actual_class'

In [12]:
import os
import cv2
import torch
import pandas as pd
from transformers import VideoMAEFeatureExtractor, TimesformerForVideoClassification
from torchvision import transforms
from tqdm import tqdm

# Initialize model, feature extractor
model_name = "facebook/timesformer-base-finetuned-k400"
device = "cuda" if torch.cuda.is_available() else "cpu"

feature_extractor = VideoMAEFeatureExtractor.from_pretrained(model_name)
model = TimesformerForVideoClassification.from_pretrained(model_name)
model.to(device)
model.eval()

id2label = model.config.id2label

# Define preprocessing transform
preprocess = transforms.Compose([
    transforms.ToPILImage(),
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize(mean=feature_extractor.image_mean, std=feature_extractor.image_std),
])

# Function to load and sample frames from video
def load_video_frames(video_path, num_frames=8):
    cap = cv2.VideoCapture(video_path)
    total_frames = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))
    frame_indices = list(range(0, total_frames, max(1, total_frames // num_frames)))[:num_frames]

    frames = []
    for i in range(total_frames):
        ret, frame = cap.read()
        if not ret:
            break
        if i in frame_indices:
            frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
            frame = preprocess(frame)
            frames.append(frame)
    cap.release()

    if len(frames) < num_frames:
        # Pad with last frame if not enough
        frames += [frames[-1]] * (num_frames - len(frames))

    video_tensor = torch.stack(frames).unsqueeze(0)  # Shape: (1, T, C, H, W)
    return video_tensor

# Prediction function
def predict_video_class(video_path):
    video_tensor = load_video_frames(video_path)  # Shape: (1, T, C, H, W)
    video_tensor = video_tensor.permute(0, 2, 1, 3, 4).to(device)  # (B, C, T, H, W)

    with torch.no_grad():
        outputs = model(video_tensor)
        predicted_class = torch.argmax(outputs.logits, dim=1).item()

    return id2label[predicted_class]

# Evaluate videos in folder
def evaluate_videos_in_folder(video_dir):
    results = []

    for filename in tqdm(os.listdir(video_dir)):
        if filename.endswith(".mp4"):
            video_path = os.path.join(video_dir, filename)
            predicted_class = predict_video_class(video_path)
            actual_class = filename.split("_")[0]  # Assumes "className_videoName.mp4"

            results.append({
                "filename": filename,
                "actual_class": actual_class,
                "predicted_class": predicted_class
            })

    df = pd.DataFrame(results)

    if not df.empty:
        df["is_correct"] = df["actual_class"] == df["predicted_class"]
        total = len(df)
        correct = df["is_correct"].sum()
        incorrect = total - correct
        most_misclassified = df[~df["is_correct"]]["actual_class"].value_counts().idxmax() if incorrect > 0 else None

        print("\n=== Prediction Summary ===")
        print(df)

        print("\n=== Statistics ===")
        print(f"🎞️ Total videos processed     : {total}")
        print(f"✅ Correct predictions        : {correct}")
        print(f"❌ Incorrect predictions      : {incorrect}")
        if most_misclassified:
            print(f"⚠️ Most misclassified class   : {most_misclassified}")
    else:
        print("❌ No valid videos found or predictions made.")

    return df

# Run the evaluation
if __name__ == "__main__":
    video_folder = r"F:\quest_digiflex\exp\data\archive_1\test\test"  # Change to your folder path
    df_results = evaluate_videos_in_folder(video_folder)


C:\Users\prana\AppData\Local\Programs\Python\Python312\Lib\site-packages\transformers\models\videomae\feature_extraction_videomae.py:28: FutureWarning: The class VideoMAEFeatureExtractor is deprecated and will be removed in version 5 of Transformers. Please use VideoMAEImageProcessor instead.
  warnings.warn(


config.json:   0%|          | 0.00/22.7k [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/486M [00:00<?, ?B/s]

100%|██████████████████████████████████████████████████████████████████████████████████████████| 22/22 [00:00<?, ?it/s]

❌ No valid videos found or predictions made.


model.safetensors:   0%|          | 0.00/486M [00:00<?, ?B/s]

In [14]:
import os
import cv2
import torch
import numpy as np
from transformers import VideoMAEImageProcessor, TimesformerForVideoClassification
from tqdm import tqdm

# Set device
device = "cuda" if torch.cuda.is_available() else "cpu"

# Load model and image processor
model_name = "facebook/timesformer-base-finetuned-k400"
processor = VideoMAEImageProcessor.from_pretrained(model_name)
model = TimesformerForVideoClassification.from_pretrained(model_name).to(device)
model.eval()

# Function to extract frames from video
def load_video(path, num_frames=16, size=224):
    cap = cv2.VideoCapture(path)
    total_frames = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))

    if total_frames < num_frames:
        print(f"⚠️ Skipping '{path}': not enough frames ({total_frames})")
        return None

    # Uniform sampling
    frame_indices = np.linspace(0, total_frames - 1, num=num_frames, dtype=int)
    frames = []

    for i in range(total_frames):
        ret, frame = cap.read()
        if not ret:
            break
        if i in frame_indices:
            frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
            frame = cv2.resize(frame, (size, size))
            frames.append(frame)

    cap.release()

    if len(frames) != num_frames:
        print(f"⚠️ Skipping '{path}': only got {len(frames)} frames")
        return None

    return np.stack(frames)

# Directory with videos
video_folder = r"F:\quest_digiflex\exp\data\archive_1\test\test"  # Update this!
video_paths = [os.path.join(video_folder, f) for f in os.listdir(video_folder) if f.endswith((".mp4", ".avi"))]

# Inference
results = []
for video_path in tqdm(video_paths):
    frames = load_video(video_path)
    if frames is None:
        continue

    inputs = processor(list(frames), return_tensors="pt")
    inputs = {k: v.unsqueeze(0).to(device) for k, v in inputs.items()}

    with torch.no_grad():
        outputs = model(**inputs)
        logits = outputs.logits
        predicted_class = logits.argmax(-1).item()

    results.append((os.path.basename(video_path), predicted_class))

if results:
    print("✅ Predictions:")
    for name, pred in results:
        print(f"{name} => Class {pred}")
else:
    print("❌ No valid videos found or predictions made.")


0it [00:00, ?it/s]

❌ No valid videos found or predictions made.


In [15]:
def load_video(path, num_frames=16, size=224):
    cap = cv2.VideoCapture(path)
    total_frames = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))
    print(f"📹 Loading {path} | Total frames: {total_frames}")

    if total_frames < num_frames:
        print(f"⚠️ Skipping '{path}': not enough frames ({total_frames})")
        return None

    frame_indices = np.linspace(0, total_frames - 1, num=num_frames, dtype=int)
    frames = []

    for i in range(total_frames):
        ret, frame = cap.read()
        if not ret:
            break
        if i in frame_indices:
            frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
            frame = cv2.resize(frame, (size, size))
            frames.append(frame)

    cap.release()

    if len(frames) != num_frames:
        print(f"⚠️ Skipping '{path}': only got {len(frames)} frames")
        return None

    return np.stack(frames)


In [19]:
load_video(r"F:\quest_digiflex\exp\test\test\barbell biceps curl\barbell biceps curl_1.mp4")

📹 Loading F:\quest_digiflex\exp\test\test\barbell biceps curl\barbell biceps curl_1.mp4 | Total frames: 795


array([[[[  2,   0,   8],
         [  2,   0,   8],
         [  2,   0,   8],
         ...,
         [ 23,  18,  34],
         [ 23,  18,  34],
         [ 23,  18,  34]],

        [[  2,   0,   8],
         [  2,   0,   8],
         [  2,   0,   8],
         ...,
         [ 23,  18,  34],
         [ 23,  18,  34],
         [ 23,  18,  34]],

        [[  2,   0,   7],
         [  2,   0,   7],
         [  2,   0,   7],
         ...,
         [ 23,  18,  34],
         [ 23,  18,  34],
         [ 23,  18,  34]],

        ...,

        [[108,  82,  71],
         [110,  84,  71],
         [112,  86,  73],
         ...,
         [141, 106,  84],
         [142, 107,  85],
         [143, 108,  86]],

        [[110,  84,  74],
         [110,  84,  72],
         [112,  87,  73],
         ...,
         [149, 114,  89],
         [149, 114,  89],
         [149, 114,  89]],

        [[110,  84,  74],
         [113,  87,  75],
         [115,  89,  75],
         ...,
         [163, 120,  94],
        

In [22]:
import os
import cv2

video_folder = r"F:\quest_digiflex\exp\test"
video_extensions = ['.mp4', '.avi', '.mov', '.mkv']

print("Listing all files in:", video_folder)
files = os.listdir(video_folder)
print(files)

for filename in files:
    print("Checking:", filename)
    if any(filename.lower().endswith(ext) for ext in video_extensions):
        path = os.path.join(video_folder, filename)
        print(f"▶ Processing video: {path}")
        cap = cv2.VideoCapture(path)
        if not cap.isOpened():
            print(f"⚠️ Failed to open: {path}")
            continue
        frame_count = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))
        print(f"✅ Frame count: {frame_count}")
        cap.release()


Listing all files in: F:\quest_digiflex\exp\test
['test']
Checking: test


In [24]:
import os
import torch
import cv2
import numpy as np
from transformers import TimesformerForVideoClassification, AutoImageProcessor
from tqdm import tqdm

# Load the model and feature extractor (AutoImageProcessor replaced deprecated TimesformerFeatureExtractor)
model_name = "facebook/timesformer-base-finetuned-k400"
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

print("Loading model and processor...")
processor = AutoImageProcessor.from_pretrained(model_name)
model = TimesformerForVideoClassification.from_pretrained(model_name).to(device)
model.eval()

# Path to root folder containing exercise folders
root_folder = r"F:\quest_digiflex\exp\tf\videomae-finetuned\test"

video_extensions = ['.mp4', '.avi', '.mov', '.mkv']

# Helper to read video frames
def read_video_frames(video_path, num_frames=8):
    cap = cv2.VideoCapture(video_path)
    if not cap.isOpened():
        print(f"❌ Cannot open video: {video_path}")
        return None

    total_frames = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))
    if total_frames < num_frames:
        print(f"⚠️ Video {video_path} has fewer than {num_frames} frames ({total_frames} frames). Skipping.")
        cap.release()
        return None

    # Sample frames uniformly
    frame_idxs = np.linspace(0, total_frames - 1, num_frames).astype(int)

    frames = []
    for i in range(total_frames):
        ret, frame = cap.read()
        if not ret:
            break
        if i in frame_idxs:
            # Convert BGR to RGB
            frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
            frames.append(frame)

    cap.release()
    if len(frames) != num_frames:
        print(f"⚠️ Warning: Collected {len(frames)} frames instead of {num_frames}. Skipping video.")
        return None

    return frames

def predict_video(video_path):
    frames = read_video_frames(video_path)
    if frames is None:
        return None

    # Process frames with processor
    inputs = processor(frames, return_tensors="pt")

    # Move inputs to device
    for k, v in inputs.items():
        inputs[k] = v.to(device)

    with torch.no_grad():
        outputs = model(**inputs)
        logits = outputs.logits
        predicted_class_idx = logits.argmax(-1).item()

    # Get label from model config
    label = model.config.id2label[predicted_class_idx]
    return label

# Iterate over exercises and videos
for exercise_folder in os.listdir(root_folder):
    exercise_path = os.path.join(root_folder, exercise_folder)
    if not os.path.isdir(exercise_path):
        continue

    print(f"\n➡️ Processing exercise folder: {exercise_folder}")

    for filename in os.listdir(exercise_path):
        if any(filename.lower().endswith(ext) for ext in video_extensions):
            video_path = os.path.join(exercise_path, filename)
            print(f"  ▶ Video: {filename}")

            label = predict_video(video_path)
            if label is not None:
                print(f"    🎯 Predicted action: {label}")
            else:
                print(f"    ❌ Skipped video due to insufficient frames or error.")


Loading model and processor...

➡️ Processing exercise folder: barbell biceps curl
  ▶ Video: barbell biceps curl_1.mp4
    🎯 Predicted action: front raises
  ▶ Video: video1.mp4
    🎯 Predicted action: front raises

➡️ Processing exercise folder: bench press
  ▶ Video: bench press_1.mp4
    🎯 Predicted action: bench pressing
  ▶ Video: bench press_2.mp4
    🎯 Predicted action: bench pressing
  ▶ Video: video2.mp4
    🎯 Predicted action: bench pressing

➡️ Processing exercise folder: chest fly machine
  ▶ Video: chest fly machine_1.mp4
    🎯 Predicted action: exercising arm
  ▶ Video: chest fly machine_2.mp4
    🎯 Predicted action: exercising arm
  ▶ Video: chest fly machine_3.mp4
    🎯 Predicted action: stretching leg
  ▶ Video: video3.mp4
    🎯 Predicted action: front raises

➡️ Processing exercise folder: deadlift
  ▶ Video: deadlift_1.mp4
    🎯 Predicted action: deadlifting
  ▶ Video: video4.mp4
    🎯 Predicted action: deadlifting

➡️ Processing exercise folder: decline bench press

KeyboardInterrupt: 

In [25]:
import os
import torch
import cv2
import numpy as np
from transformers import TimesformerForVideoClassification, AutoImageProcessor
from tqdm import tqdm

# Load the model and processor
model_name = "facebook/timesformer-base-finetuned-k400"
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

print("Loading model and processor...")
processor = AutoImageProcessor.from_pretrained(model_name)
model = TimesformerForVideoClassification.from_pretrained(model_name).to(device)
model.eval()

# Path to root folder containing exercise folders
root_folder = r"F:\quest_digiflex\exp\tf\videomae-finetuned\test"
video_extensions = ['.mp4', '.avi', '.mov', '.mkv']

# Helper to read video frames
def read_video_frames(video_path, num_frames=8):
    cap = cv2.VideoCapture(video_path)
    if not cap.isOpened():
        print(f"❌ Cannot open video: {video_path}")
        return None
    total_frames = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))
    if total_frames < num_frames:
        print(f"⚠️ Video {video_path} has fewer than {num_frames} frames ({total_frames} frames). Skipping.")
        cap.release()
        return None
    frame_idxs = np.linspace(0, total_frames - 1, num_frames).astype(int)
    frames = []
    for i in range(total_frames):
        ret, frame = cap.read()
        if not ret:
            break
        if i in frame_idxs:
            frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
            frames.append(frame)
    cap.release()
    if len(frames) != num_frames:
        print(f"⚠️ Warning: Collected {len(frames)} frames instead of {num_frames}. Skipping video.")
        return None
    return frames

def predict_video(video_path):
    frames = read_video_frames(video_path)
    if frames is None:
        return None
    inputs = processor(frames, return_tensors="pt")
    for k, v in inputs.items():
        inputs[k] = v.to(device)
    with torch.no_grad():
        outputs = model(**inputs)
        logits = outputs.logits
        predicted_class_idx = logits.argmax(-1).item()
        label = model.config.id2label[predicted_class_idx]
    return label

# Counters
total_videos = 0
correct_predictions = 0
incorrect_predictions = 0
# Track mismatches: ground_truth_class -> count
mismatch_counter = {}

print(f"\nProcessing videos from root folder: {root_folder}")

for exercise_folder in sorted(os.listdir(root_folder)):
    exercise_path = os.path.join(root_folder, exercise_folder)
    if not os.path.isdir(exercise_path):
        continue

    print(f"\n➡️ Processing exercise folder: {exercise_folder}")

    for filename in sorted(os.listdir(exercise_path)):
        if not any(filename.lower().endswith(ext) for ext in video_extensions):
            continue
        video_path = os.path.join(exercise_path, filename)
        print(f" ▶ Video: {filename}")

        predicted_label = predict_video(video_path)
        if predicted_label is None:
            print(" ❌ Skipped video due to insufficient frames or error.")
            continue

        total_videos += 1
        print(f" 🎯 Predicted action: {predicted_label}")

        # Compare predicted label with folder name (ground truth)
        if predicted_label.lower() == exercise_folder.lower():
            correct_predictions += 1
        else:
            incorrect_predictions += 1
            mismatch_counter[exercise_folder] = mismatch_counter.get(exercise_folder, 0) + 1

# Summary report
print("\n\n====== Summary ======")
print(f"Total videos processed: {total_videos}")
print(f"Correct predictions: {correct_predictions}")
print(f"Incorrect predictions: {incorrect_predictions}")

if mismatch_counter:
    print("\nMost mismatched classes:")
    # Sort by count desc
    sorted_mismatches = sorted(mismatch_counter.items(), key=lambda x: x[1], reverse=True)
    for cls, count in sorted_mismatches:
        print(f" - {cls}: {count} mismatches")
else:
    print("\nNo mismatches found, great!")



Loading model and processor...

Processing videos from root folder: F:\quest_digiflex\exp\tf\videomae-finetuned\test

➡️ Processing exercise folder: barbell biceps curl
 ▶ Video: barbell biceps curl_1.mp4
 🎯 Predicted action: front raises
 ▶ Video: video1.mp4
 🎯 Predicted action: front raises

➡️ Processing exercise folder: bench press
 ▶ Video: bench press_1.mp4
 🎯 Predicted action: bench pressing
 ▶ Video: bench press_2.mp4
 🎯 Predicted action: bench pressing
 ▶ Video: video2.mp4
 🎯 Predicted action: bench pressing

➡️ Processing exercise folder: chest fly machine
 ▶ Video: chest fly machine_1.mp4
 🎯 Predicted action: exercising arm
 ▶ Video: chest fly machine_2.mp4
 🎯 Predicted action: exercising arm
 ▶ Video: chest fly machine_3.mp4
 🎯 Predicted action: stretching leg
 ▶ Video: video3.mp4
 🎯 Predicted action: front raises

➡️ Processing exercise folder: deadlift
 ▶ Video: deadlift_1.mp4
 🎯 Predicted action: deadlifting
 ▶ Video: video4.mp4
 🎯 Predicted action: deadlifting

➡️ Proce

In [26]:
# Summary report
print("\n\n====== Summary ======")
print(f"Total videos processed: {total_videos}")
print(f"Correct predictions: {correct_predictions}")
print(f"Incorrect predictions: {incorrect_predictions}")
accuracy = correct_predictions / total_videos if total_videos > 0 else 0
print(f"Accuracy: {accuracy:.2%}")

if mismatch_counter:
    print("\nMost mismatched classes:")
    # Sort by count desc
    sorted_mismatches = sorted(mismatch_counter.items(), key=lambda x: x[1], reverse=True)
    for cls, count in sorted_mismatches:
        print(f" - {cls}: {count} mismatches")
else:
    print("\nNo mismatches found, great!")




====== Summary ======
Total videos processed: 61
Correct predictions: 3
Incorrect predictions: 58
Accuracy: 4.92%

Most mismatched classes:
 - chest fly machine: 4 mismatches
 - bench press: 3 mismatches
 - decline bench press: 3 mismatches
 - hammer curl: 3 mismatches
 - hip thrust: 3 mismatches
 - incline bench press: 3 mismatches
 - lat pulldown: 3 mismatches
 - leg extension: 3 mismatches
 - leg raises: 3 mismatches
 - pull Up: 3 mismatches
 - romanian deadlift: 3 mismatches
 - russian twist: 3 mismatches
 - shoulder press: 3 mismatches
 - t bar row: 3 mismatches
 - tricep dips: 3 mismatches
 - barbell biceps curl: 2 mismatches
 - deadlift: 2 mismatches
 - lateral raise: 2 mismatches
 - plank: 2 mismatches
 - push-up: 2 mismatches
 - tricep Pushdown: 2 mismatches
